In [1]:
import pandas as pd

In [2]:
weather = pd.read_csv('weather.csv', index_col = 'DATE')

In [3]:
weather = weather.sort_index()

Exploration and Preprocessing

In [4]:
# % of null values in each column
null_pct = weather.isna().sum()/weather.shape[0]
null_pct

,0
STATION,0.000000
NAME,0.000000
PRCP,0.057625
SNWD,0.175524
TAVG,0.860672
TMAX,0.077978
TMIN,0.078451


In [5]:
# filling the missing values of average temperature with the mean of min and max temperature of that day
weather['TAVG'] = weather['TAVG'].fillna(
    (weather[['TMIN', 'TMAX']].mean(axis=1))
)

# filling the missing values of snow depth (SNWD) using forward fill - the previous value gets carried over to the nan cell,
# makes sense because if the snow depth was 0 yesterday, it will most likely be 0 today as well.
weather['SNWD'] = weather['SNWD'].ffill()


# removing the remaining values which amount to less than 8% for TMAX and TMIN and 5.7% for PRCP, along with 3020 records of TAVG
weather = weather.dropna()
print(weather.isna().sum())
weather.shape


STATION    0
NAME       0
PRCP       0
SNWD       0
TAVG       0
TMAX       0
TMIN       0
dtype: int64


(126581, 7)

In [6]:
weather.dtypes

,0
STATION,object
NAME,object
PRCP,float64
SNWD,float64
TAVG,float64
TMAX,float64
TMIN,float64


In [7]:
# converting index - date - to datetime datatype from object
weather.index = pd.to_datetime(weather.index)
weather

,STATION,NAME,PRCP,SNWD,TAVG,TMAX,TMIN
DATE,,,,,,,
1990-01-01,GME00130786,"BERGE, GM",0.01,0.0,29.0,30.0,28.0
1990-01-01,GME00111445,"BERLIN TEMPELHOF, GM",0.00,0.0,29.0,31.0,28.0
1990-01-01,GM000003319,"BERLIN DAHLEM, GM",0.01,0.0,29.0,31.0,28.0
1990-01-01,GME00127930,"BERLIN SCHONEFELD, GM",0.00,0.0,29.5,31.0,28.0
1990-01-01,GME00127762,"BERLIN LICHTENRADE, GM",0.00,0.0,30.0,31.0,29.0
...,...,...,...,...,...,...,...
2025-10-31,GME00127822,"BERLIN MARZAHN, GM",0.00,0.0,47.0,56.0,38.0
2025-10-31,GM000003342,"POTSDAM, GM",0.00,0.0,46.5,55.0,38.0
2025-10-31,GME00127438,"BERLIN BUCH, GM",0.00,0.0,45.5,55.0,36.0


In [8]:
# creating target column - correct prediction for any day is the actual tempe from the next day - in a way it's labeling data
weather['TARGET_TMAX'] = weather.shift(-1)['TMAX']

# the last row of these columns will have NaN as there is no next record to shift up - so just use forward fill
weather.ffill()

,STATION,NAME,PRCP,SNWD,TAVG,TMAX,TMIN,TARGET_TMAX
DATE,,,,,,,,
1990-01-01,GME00130786,"BERGE, GM",0.01,0.0,29.0,30.0,28.0,31.0
1990-01-01,GME00111445,"BERLIN TEMPELHOF, GM",0.00,0.0,29.0,31.0,28.0,31.0
1990-01-01,GM000003319,"BERLIN DAHLEM, GM",0.01,0.0,29.0,31.0,28.0,31.0
1990-01-01,GME00127930,"BERLIN SCHONEFELD, GM",0.00,0.0,29.5,31.0,28.0,31.0
1990-01-01,GME00127762,"BERLIN LICHTENRADE, GM",0.00,0.0,30.0,31.0,29.0,31.0
...,...,...,...,...,...,...,...,...
2025-10-31,GME00127822,"BERLIN MARZAHN, GM",0.00,0.0,47.0,56.0,38.0,55.0
2025-10-31,GM000003342,"POTSDAM, GM",0.00,0.0,46.5,55.0,38.0,55.0
2025-10-31,GME00127438,"BERLIN BUCH, GM",0.00,0.0,45.5,55.0,36.0,56.0


In [9]:
# we're using RidgeRegression, as it is an ml model that deals with much of the multi-correlations
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 0.1)
# alpha accounts for how much the parameters are shrunk to account for multi-correlation, 0.1 is a good rule of thumb

In [10]:
predictors = weather.columns[~weather.columns.isin(['NAME', 'STATION', 'TARGET_TMAX'])]
predictors

Index(['PRCP', 'SNWD', 'TAVG', 'TMAX', 'TMIN'], dtype='object')

In [19]:
# since it is a time series data, we cannot do simple cross-validation - we cannot use future data to predict the past data
# this is a replacement function of that

# start: all the records over say 10 years - total records (126581) divided by 10,
# step: 90 days, so the model predicts for every 90 days then moves on to the next 90 days
def backtest(weather, model, predictors, start = 36500, step = 90):
  all_predictions = []
  for i in range(start, weather.shape[0], step):
    train = weather.iloc[: i, :]
    test = weather.iloc[i: (i + step), :]
    model.fit(train[predictors], train['TARGET_TMAX'])
    preds = model.predict(test[predictors])

    preds = pd.Series(preds, index = test.index)
    combined = pd.concat([test['TARGET_TMAX'], preds], axis = 1)

    combined.columns = ['Actual', 'Prediction']
    combined['Diff'] = (combined['Actual'] - combined['Prediction']).abs()

    all_predictions.append(combined)
  return pd.concat(all_predictions)



In [20]:
# actually making predictions
prediction = backtest(weather, ridge, predictors)
prediction

,Actual,Prediction,Diff
DATE,,,
1999-08-28,74.0,73.968809,0.031191
1999-08-28,74.0,73.939505,0.060495
1999-08-29,74.0,73.734378,0.265622
1999-08-29,72.0,73.851594,1.851594
1999-08-29,72.0,71.880090,0.119910
...,...,...,...
2025-10-31,55.0,55.942848,0.942848
2025-10-31,55.0,54.974205,0.025795
2025-10-31,56.0,54.910766,1.089234


In [24]:
print(prediction)
prediction.ffill(inplace = True)

            Actual  Prediction      Diff
DATE                                    
1999-08-28    74.0   73.968809  0.031191
1999-08-28    74.0   73.939505  0.060495
1999-08-29    74.0   73.734378  0.265622
1999-08-29    72.0   73.851594  1.851594
1999-08-29    72.0   71.880090  0.119910
...            ...         ...       ...
2025-10-31    55.0   55.942848  0.942848
2025-10-31    55.0   54.974205  0.025795
2025-10-31    56.0   54.910766  1.089234
2025-10-31    53.0   55.942848  2.942848
2025-10-31     NaN   53.036919       NaN

[90081 rows x 3 columns]


In [25]:
# accuracy metric -
from sklearn.metrics import mean_absolute_error
mean_absolute_error(prediction['Actual'], prediction['Prediction'])
# prediction['Actual'].isna()

1.4737303305490315

In [26]:
prediction['Diff'].mean()

np.float64(1.4737625896171582)